In [ ]:
from src.sampling.images import plot_frame_count_distributions

samples_root_dir = "data/samples"

plot_frame_count_distributions(samples_root_dir)

In [ ]:
from sota.model import SOTA

sota = SOTA("data", "yolo11m-cls")
sota.initialize_model()
sota.train_model(optimizer="AdamW", lr0=0.0005)


#metrics = model.val(data="data/img/techniques/val")

In [ ]:
from sota.model import SOTA

sota = SOTA("data", "yolo11n-cls")
sota.initialize_model()
sota.train_model()

In [ ]:
from src.sota.model import SOTA

sota = SOTA("data", "yolo11m-balance-50-155", dataset_name="techniques_balanced")
sota.initialize_model("yolo11m-cls")
sota.train_model()

In [3]:
from src.sota.model import SOTA

sota = SOTA("data", "yolo11n-balance-50-155", dataset_name="techniques_balanced")
sota.initialize_model("yolo11n-cls")
#sota.train_model()
sota.test_model()

loading the model 'yolo11n-balance-50-155' with the weights at 'data/runs/sota/yolo11n-balance-50-155/train5/weights/best.pt'
Make sure to swap val and test splits for techniques_balanced, otherwise validation data will be used.
Ultralytics 8.3.78 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA TITAN RTX, 24576MiB)
YOLO11n-cls summary (fused): 47 layers, 1,534,991 parameters, 0 gradients, 3.2 GFLOPs
train: /mnt/c/Projects/climbing-technique-detector/data/img/techniques_balanced/train... found 1893 images in 7 classes ✅ 
val: /mnt/c/Projects/climbing-technique-detector/data/img/techniques_balanced/val... found 416 images in 7 classes ✅ 
test: /mnt/c/Projects/climbing-technique-detector/data/img/techniques_balanced/test... found 894 images in 7 classes ✅ 


val: Scanning /mnt/c/Projects/climbing-technique-detector/data/img/techniques_balanced/val... 416 images, 0 corrupt: 100%|██████████| 416/416 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 26/26 [00:13<00:00,  1.95it/s]


                   all      0.892      0.998
Speed: 1.0ms preprocess, 21.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /mnt/c/Projects/HPE-POC/runs/classify/val12


In [5]:
from src.hpe_dnn.model import HpeDnn

hpednn = HpeDnn("data", "arch1")
hpednn.initialize_model()
hpednn.train_model()

loading the model 'arch1' from 'data/runs/hpe_dnn/arch1/train1/models/epoch_02__val_accuracy_0.7152.keras'
     NOSE_x    NOSE_y    NOSE_z  NOSE_visibility  LEFT_SHOULDER_x  \
0  0.631847  0.439228 -0.112943         0.967189         0.584897   
1  0.544295  0.383693  0.030225         0.970460         0.534586   
2  0.522651  0.319287  0.059222         0.973314         0.477302   
3  0.509914  0.345323  0.076138         0.975976         0.472843   
4  0.520337  0.300978  0.016012         0.978329         0.485842   

   LEFT_SHOULDER_y  LEFT_SHOULDER_z  LEFT_SHOULDER_visibility  LEFT_ELBOW_x  \
0         0.497950        -0.257367                  0.999389      0.505804   
1         0.445929        -0.061585                  0.999447      0.459553   
2         0.356460        -0.093307                  0.999499      0.423448   
3         0.384584        -0.048136                  0.999547      0.429408   
4         0.349062        -0.114536                  0.999589      0.421399   

   

In [ ]:
%load_ext tensorboard

from src.hpe_dnn.model import HpeDnn

hpednn = HpeDnn("data", "arch1_balanced", "techniques_balanced")
hpednn.initialize_model()
hpednn.train_model()

In [4]:
from src.hpe_dnn.model import HpeDnn, DnnArch

hpednn = HpeDnn("data", "arch2_balanced", "techniques_balanced")
hpednn.initialize_model(DnnArch.ARCH2)
hpednn.train_model()

loading the model 'arch2_balanced' from 'data/runs/hpe_dnn/arch2_balanced/train2/models/epoch_12__val_accuracy_0.4639.keras'
     NOSE_x    NOSE_y    NOSE_z  NOSE_visibility  LEFT_SHOULDER_x  \
0  0.547458  0.463912 -0.324645         0.958448         0.555777   
1  0.529612  0.401783 -0.133231         0.962421         0.546691   
2  0.521964  0.327832 -0.046612         0.963201         0.486264   
3  0.507897  0.377151 -0.066129         0.964212         0.478969   
4  0.517631  0.282593  0.044080         0.967762         0.474001   

   LEFT_SHOULDER_y  LEFT_SHOULDER_z  LEFT_SHOULDER_visibility  LEFT_ELBOW_x  \
0         0.487867        -0.281746                  0.998441      0.490354   
1         0.480926        -0.178477                  0.998549      0.466458   
2         0.366174        -0.130556                  0.998679      0.427865   
3         0.402826        -0.136893                  0.998766      0.430541   
4         0.337471        -0.096800                  0.998888    

In [3]:
from src.hpe_dnn.model import HpeDnn, DnnArch

hpednn = HpeDnn("data", "arch3_balanced", "techniques_balanced")
hpednn.initialize_model(DnnArch.ARCH3)
hpednn.train_model()

loading the model 'arch3_balanced' from 'data/runs/hpe_dnn/arch3_balanced/train2/models/epoch_19__val_accuracy_0.5096.keras'
     NOSE_x    NOSE_y    NOSE_z  NOSE_visibility  LEFT_SHOULDER_x  \
0  0.547458  0.463912 -0.324645         0.958448         0.555777   
1  0.529612  0.401783 -0.133231         0.962421         0.546691   
2  0.521964  0.327832 -0.046612         0.963201         0.486264   
3  0.507897  0.377151 -0.066129         0.964212         0.478969   
4  0.517631  0.282593  0.044080         0.967762         0.474001   

   LEFT_SHOULDER_y  LEFT_SHOULDER_z  LEFT_SHOULDER_visibility  LEFT_ELBOW_x  \
0         0.487867        -0.281746                  0.998441      0.490354   
1         0.480926        -0.178477                  0.998549      0.466458   
2         0.366174        -0.130556                  0.998679      0.427865   
3         0.402826        -0.136893                  0.998766      0.430541   
4         0.337471        -0.096800                  0.998888    

In [1]:
from src.hpe_dnn.model import HpeDnn, DnnArch

hpednn = HpeDnn("data", "arch1_balanced_not_norm", "techniques_balanced")
hpednn.initialize_model(DnnArch.ARCH1, normalize=False)
hpednn.train_model()

2025-06-06 10:00:11.769271: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749196812.999186   48654 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749196813.317693   48654 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-06 10:00:16.412207: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


loading a fresh model 'arch1_balanced_not_norm'
     NOSE_x    NOSE_y    NOSE_z  NOSE_visibility  LEFT_SHOULDER_x  \
0  0.547458  0.463912 -0.324645         0.958448         0.555777   
1  0.529612  0.401783 -0.133231         0.962421         0.546691   
2  0.521964  0.327832 -0.046612         0.963201         0.486264   
3  0.507897  0.377151 -0.066129         0.964212         0.478969   
4  0.517631  0.282593  0.044080         0.967762         0.474001   

   LEFT_SHOULDER_y  LEFT_SHOULDER_z  LEFT_SHOULDER_visibility  LEFT_ELBOW_x  \
0         0.487867        -0.281746                  0.998441      0.490354   
1         0.480926        -0.178477                  0.998549      0.466458   
2         0.366174        -0.130556                  0.998679      0.427865   
3         0.402826        -0.136893                  0.998766      0.430541   
4         0.337471        -0.096800                  0.998888      0.422533   

   LEFT_ELBOW_y  ...  LEFT_THUMB_TIP_x  LEFT_THUMB_TIP_y  LEFT

I0000 00:00:1749196865.930456   48654 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22276 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:21:00.0, compute capability: 7.5
2025-06-06 10:01:12.411467: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-06-06 10:01:12.752430: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-06-06 10:01:13.417026: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-06-06 10:01:14.752521: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-06-06 10:01:17.471972: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-0

     NOSE_x    NOSE_y    NOSE_z  NOSE_visibility  LEFT_SHOULDER_x  \
0  0.547458  0.463912 -0.324645         0.958448         0.555777   
1  0.529612  0.401783 -0.133231         0.962421         0.546691   
2  0.521964  0.327832 -0.046612         0.963201         0.486264   
3  0.507897  0.377151 -0.066129         0.964212         0.478969   
4  0.517631  0.282593  0.044080         0.967762         0.474001   

   LEFT_SHOULDER_y  LEFT_SHOULDER_z  LEFT_SHOULDER_visibility  LEFT_ELBOW_x  \
0         0.487867        -0.281746                  0.998441      0.490354   
1         0.480926        -0.178477                  0.998549      0.466458   
2         0.366174        -0.130556                  0.998679      0.427865   
3         0.402826        -0.136893                  0.998766      0.430541   
4         0.337471        -0.096800                  0.998888      0.422533   

   LEFT_ELBOW_y  ...  LEFT_THUMB_TIP_x  LEFT_THUMB_TIP_y  LEFT_THUMB_TIP_z  \
0      0.434804  ...            

I0000 00:00:1749196909.817802   49583 service.cc:148] XLA service 0x7f51c803ec20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1749196909.819439   49583 service.cc:156]   StreamExecutor device (0): NVIDIA TITAN RTX, Compute Capability 7.5
2025-06-06 10:01:50.236854: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1749196911.201875   49583 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1749196916.717372   49583 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.2288 - loss: 0.5300
Epoch 1: val_accuracy improved from -inf to 0.28846, saving model to data/runs/hpe_dnn/arch1_balanced_not_norm/train1/models/epoch_01__val_accuracy_0.2885.keras
60/60 ━━━━━━━━━━━━━━━━━━━━ 23s 192ms/step - accuracy: 0.2295 - loss: 0.5288 - val_accuracy: 0.2885 - val_loss: 0.4018
Epoch 2/20
58/60 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.3855 - loss: 0.3680
Epoch 2: val_accuracy improved from 0.28846 to 0.38221, saving model to data/runs/hpe_dnn/arch1_balanced_not_norm/train1/models/epoch_02__val_accuracy_0.3822.keras
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - accuracy: 0.3877 - loss: 0.3671 - val_accuracy: 0.3822 - val_loss: 0.3575
Epoch 3/20
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5685 - loss: 0.2966
Epoch 3: val_accuracy improved from 0.38221 to 0.41827, saving model to data/runs/hpe_dnn/arch1_balanced_not_norm/train1/models/epoch_03__val_accuracy_0.4183.keras
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 65m

In [ ]:
type(func)

In [ ]:
%tensorboard --logdir data/runs/hpe_dnn/arch1_balanced/train1/logs/train